## Imports

In [2]:
#Import necessary libraries and functions
#!pip install opencv-python
import requests 
from bs4 import BeautifulSoup 
import re
import json
#import urllib
import os
import urllib.request
import cv2
import json
import scipy.misc
import pandas as pd
from undistort_field import *
from pass_detection import *
import numpy as np
from glob import glob
import time

## Scrape the Charts

In [ ]:
teams = ["arizona-cardinals",
	"atlanta-falcons",
	"baltimore-ravens",
	"buffalo-bills",
	"carolina-panthers",
	"chicago-bears",
	"cincinnati-bengals",
	"cleveland-browns",
	"dallas-cowboys",
	"denver-broncos",
	"detroit-lions",
	"green-bay-packers",
	"houston-texans",
	"indianapolis-colts",
	"jacksonville-jaguars",
	"kansas-city-chiefs",
	"los-angeles-chargers",
	"los-angeles-rams",
	"miami-dolphins",
	"minnesota-vikings",
	"new-england-patriots",
	"new-orleans-saints",
	"new-york-giants",
	"new-york-jets",
	"oakland-raiders",
	"philadelphia-eagles",
	"pittsburgh-steelers",
	"san-francisco-49ers",
	"seattle-seahawks",
	"tampabay-buccaneers",
	"tennessee-titans",
	"washington-redskins"
]

# seasons = ["2017","2018","2019"]
seasons = ['2019']
weeks = ["1", "2", "3", "4", "5",
         "6", "7", "8", "9", "10",
         "11", "12", "13",
         '14', "15", "16", "17",
         "wild-card", "divisional",
         "conference", "super-bowl"]

pattern = re.compile("charts")

print("Scraping images and html data...")

for team in teams:
	for season in seasons:
		print(team, "\t", season)
		for week in weeks:
			URL = "https://nextgenstats.nfl.com/charts/list/route/" + team + "/" + season + "/" + week
			try:
				r = requests.get(URL)
			except:
				print(URL + '  is giving an error. Skipping...')    
				continue
			soup = BeautifulSoup(r.content, "html.parser")

			script = soup.find_all("script", text=pattern)
			if len(script)==0:
				print(URL + ' is giving an error. Skipping...')    
				continue
            
			contains_charts = json.loads(str(script[0])[33:-131])

			if (len(contains_charts["charts"]["charts"]) != 0):
				for chart in contains_charts["charts"]["charts"]['charts']:

					name = chart["lastName"] + "_" + chart["firstName"] + "_" + chart["position"]
					chart["team"] = team

					folder = str("Route_Charts" + os.sep + team + os.sep + season + os.sep + week + os.sep)
					img_folder = folder + "images" + os.sep
					data_folder = folder + "data" + os.sep

					if not os.path.exists(img_folder):
						os.makedirs(img_folder)
					if not os.path.exists(data_folder):
						os.makedirs(data_folder)

					img_file = img_folder + name + ".jpeg"
					url = "https:" + chart["extraLargeImg"]
					try:
						urllib.request.urlretrieve(url, img_file)
					except:
						print(URL + '  is giving an error. Skipping...') 
						continue
# 					urllib.request.urlretrieve(url, img_file)

					data_file = data_folder + name + ".txt"
					with open(data_file, 'w') as datafile: 
						json.dump(chart, datafile)

print("Done.")


## Clean the charts

In [8]:
#Clean the images from the charts ('clean.py')
def new_image(image):

	img_name = image.split(os.sep)[-1].split(".")[0]
	img = cv2.imread(image)

	if (img.shape[0:2] == (1200, 1200)):
		crop_img = img[0:680, 0:1200]
		temp_name = img_name + "_temp.jpg"
		cv2.imwrite(temp_name, crop_img)
	else:
		print("Image must be of size (1200, 1200)")
		return

	clean_img = clean_field(temp_name)
	write_path = clean_path + os.sep + os.sep.join(image.split(os.sep)[1:-1]) 
	if not os.path.exists(write_path): os.makedirs(write_path)

	if (clean_img != None):
		write_name = write_path + os.sep + img_name + ".jpeg"
		scipy.misc.imsave(write_name, clean_img)
	os.remove(temp_name)

def new_data(folder, image): 
	data_path = os.sep.join(folder.split(os.sep)[:-1]) + os.sep + "data" 
	data_file = data_path + os.sep + image.split(".")[0] + ".txt"
	if 'St.' in data_file:
		return
	new_data_path = clean_path + os.sep + os.sep.join(folder.split(os.sep)[1:-1]) + os.sep + "data" 
	new_data_file = new_data_path + os.sep + image.split(".")[0] + ".txt"

	if not os.path.exists(new_data_path): 
		os.makedirs(new_data_path)

	with open(data_file) as _file: 
		old_data = json.load(_file)
		new_data = {key: old_data[key] for key in keys}
	with open(new_data_file, "w") as _file:
		json.dump(new_data, _file)



if __name__ == '__main__':
	keys = ['receptions', 'receivingYards', 'touchdowns', 'playerNameSlug', 
		'teamId', 'playerName', 'season', 'position',
		'type', 'week', 'gameId', 'esbId', 'firstName', 'lastName', 
		'team'
	]

	clean_path = "Cleaned_Route_Charts"

	if not os.path.exists(clean_path): os.makedirs(clean_path)

	pass_chart_folders = [folder[0] for folder in os.walk("Route_Charts")]
	image_folders = [folder for folder in pass_chart_folders if folder.split(os.sep)[-1] == "images"]
	print("Cleaning images and data...")
	for folder in image_folders:
		#print folder 
		images = os.listdir(folder)
		for image in images:
# 			print(image)
# 			print(folder)
			clean_folder = 'Cleaned_Route_Charts\'' + folder[12:]            
			name = os.path.join(clean_folder,image)
			if os.path.isfile(name):
				continue
			if not image.startswith("."): 
				try:
					new_image(os.path.join(folder, image))
					new_data(folder, image)
				except:
					continue
	print("Done.")


Cleaning images and data...


C:\Users\Ethan Douglas\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Done.


## Route extraction

If anyone has ideas to improve this, please do not hesitate to contribute

In [9]:
def map_route_locations(image):
    lower_green = np.array([40,100, 100])
    upper_green = np.array([80, 255, 255])

    lower_white = np.array([230, 230, 230])
    upper_white = np.array([255, 255, 255])

    lower_gray = np.array([126, 126, 126])
    upper_gray = np.array([132, 132, 132])
    col_names = ["route_type", "x", "y"]
    route_locations = pd.DataFrame(columns = col_names)

    image = cv2.imread(image)
    row, col = image.shape[0:2]
    # Convert BGR to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Threshold the HSV image to get only green colors
    mask = cv2.inRange(hsv, lower_green, upper_green)
    mask2 = cv2.inRange(image, lower_white, upper_white)
    mask3 = cv2.inRange(image, lower_gray, upper_gray)
    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(image, image, mask=mask)
    res2 = cv2.bitwise_and(image, image, mask=mask2)
    res3 = cv2.bitwise_and(image, image, mask=mask3)
    x = np.where(res2 != 0)[0]
    y = np.where(res2 != 0)[1]
    # pairs = list(zip(x,y))
    X = np.stack((x,y),axis=1 )
    c_routes = X

    x = np.where(res != 0)[0]
    y = np.where(res != 0)[1]
    X = np.stack((x,y),axis=1 )
    yac_routes = X

    x = np.where(res3 != 0)[0]
    y = np.where(res3 != 0)[1]
    X = np.stack((x,y),axis=1 )
    inc_routes = X

    sideline = 40 # pixels
    width = 53.33 # standard width of football field
    center_x = col/2

    if col > 1370:
        _75_yd_line = 0
        LOS = 596

        _1_yd_x = float(col - sideline*2)/width
        _1_yd_y = float(LOS - _75_yd_line)/75

    else:
        _55_yd_line = 5
        LOS = 572
        _1_yd_x = float(col - sideline*2)/width
        _1_yd_y = float(LOS - _55_yd_line)/55

    route_type='COMPLETE'
    y = c_routes[:,0]
    x = c_routes[:,1]
    y_loc = (LOS - y)/_1_yd_y
    x_loc = (x - center_x)/_1_yd_x
    loc = [x_loc,y_loc]
    t_loc = np.transpose(loc)
    df = pd.DataFrame(t_loc,columns=['x','y'])
    df['route_type'] = route_type
    route_locations = route_locations.append(df, ignore_index=True)

    route_type='YAC'
    y = yac_routes[:,0]
    x = yac_routes[:,1]
    y_loc = (LOS - y)/_1_yd_y
    x_loc = (x - center_x)/_1_yd_x
    loc = [x_loc,y_loc]
    t_loc = np.transpose(loc)
    df = pd.DataFrame(t_loc,columns=['x','y'])
    df['route_type'] = route_type
    route_locations = route_locations.append(df, ignore_index=True)

    route_type='INCOMPLETE'
    y = inc_routes[:,0]
    x = inc_routes[:,1]
    y_loc = (LOS - y)/_1_yd_y
    x_loc = (x - center_x)/_1_yd_x
    loc = [x_loc,y_loc]
    t_loc = np.transpose(loc)
    df = pd.DataFrame(t_loc,columns=['x','y'])
    df['route_type'] = route_type
    route_locations = route_locations.append(df, ignore_index=True)
    
    return route_locations

In [10]:
#Translate the image data into coordinate data for each chart ('main.py')
def get_route_data(data_file): 
	"""
	Extract the number of complete passes, incomplete passes, touchdowns, and interceptions 
	from a pass chart. 
	"""
	with open(data_file) as _file: 
		data = json.load(_file)
		_file.close()
	n_receptions = data["receptions"]
	n_touchdowns = data["touchdowns"]

	return (n_receptions, n_touchdowns)

def get_image(folder, data_file):
	"""
	If cleaned image exists, return the file path to the image, 
	otherwise return none.
	"""
	images_path = os.sep.join(folder.split(os.sep)[:-1]) + os.sep + "images" 
	image_file = images_path + os.sep + data_file.split(".")[0] + ".jpeg"
	if not os.path.exists(image_file):
		return None
	else:
		return image_file

def get_game_data(data_file):
	"""
	Extract player name, team, and game ID from the data corresponding to a pass chart image.
	"""	
	with open(data_file) as _file: 
		data = json.load(_file)
		_file.close()
	name = data["firstName"] + " " + data["lastName"]
	team = data["team"]
	game_id = data["gameId"]
	position = data["position"]
	week = data_file.split(os.sep)[-3]
	return (name, team, game_id, week, position)

def write_route_locations(image, data, routes):
	"""
	Write player, team, and game information, and locations of all passes to a .csv file.
	"""
	(n_route, n_td) = get_route_data(data)
	(name,team, game_id, week, position) = get_game_data(data)
	n_total = n_route

	route_cols = ["route_type", "x", "y"]
	game_cols = ["game_id", "team", "week", "name", 'position']
	route_df = pd.DataFrame(columns = route_cols)

	if (image is None): 
		rows_route = pd.DataFrame([["NAN", None, None]]*n_route, 
			columns = route_cols)
		route_df = route_df.append([rows_route])
		game_df = pd.DataFrame([[game_id, team, week, name, position]]*route_df.shape[0], 
			columns = game_cols)
		df = pd.concat([game_df, route_df.reset_index(drop=True)], axis=1)
		routes = routes.append(df)
		return routes

	if n_total != 0: 
		rows_route = map_route_locations(image)
		route_df = route_df.append(rows_route)

	game_df = pd.DataFrame([[game_id, team, week, name, position]]*route_df.shape[0], 
		columns = game_cols)

	df = pd.concat([game_df, route_df.reset_index(drop=True)], axis=1)
	routes = routes.append(df)
	return routes


if __name__ == '__main__':

	clean_path = "Cleaned_Route_Charts"
	routes = pd.DataFrame(columns = ["game_id", "team", "week", "name", "position", "route_type", "x", "y"])

	route_chart_folders = [folder[0] for folder in os.walk(clean_path)]
	data_folders = [folder for folder in route_chart_folders if folder.split(os.sep)[-1] == "data"]

	print("Extracting route locations...")
	for folder in data_folders:
		if ('2019' in folder):
# 			print(folder)
			data = os.listdir(folder)
			print(folder)
			for data_file in data:
				if not data_file.startswith("."): 
					image = get_image(folder, data_file)
					routes = write_route_locations(image, os.path.join(folder, data_file), routes)
	routes.to_csv("route_locations_2019.csv", index=False, header=True)
	print("Done.")

Extracting route locations...
Cleaned_Route_Charts\arizona-cardinals\2018\1\data


C:\Users\Ethan Douglas\Anaconda3\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


Cleaned_Route_Charts\arizona-cardinals\2018\10\data
Cleaned_Route_Charts\arizona-cardinals\2018\11\data
Cleaned_Route_Charts\arizona-cardinals\2018\12\data
Cleaned_Route_Charts\arizona-cardinals\2018\13\data
Cleaned_Route_Charts\arizona-cardinals\2018\14\data
Cleaned_Route_Charts\arizona-cardinals\2018\15\data
Cleaned_Route_Charts\arizona-cardinals\2018\16\data
Cleaned_Route_Charts\arizona-cardinals\2018\17\data
Cleaned_Route_Charts\arizona-cardinals\2018\2\data
Cleaned_Route_Charts\arizona-cardinals\2018\3\data
Cleaned_Route_Charts\arizona-cardinals\2018\4\data
Cleaned_Route_Charts\arizona-cardinals\2018\5\data
Cleaned_Route_Charts\arizona-cardinals\2018\6\data
Cleaned_Route_Charts\arizona-cardinals\2018\7\data
Cleaned_Route_Charts\arizona-cardinals\2018\8\data
Cleaned_Route_Charts\atlanta-falcons\2018\1\data
Cleaned_Route_Charts\atlanta-falcons\2018\10\data
Cleaned_Route_Charts\atlanta-falcons\2018\11\data
Cleaned_Route_Charts\atlanta-falcons\2018\12\data
Cleaned_Route_Charts\atlanta

Cleaned_Route_Charts\detroit-lions\2018\15\data
Cleaned_Route_Charts\detroit-lions\2018\16\data
Cleaned_Route_Charts\detroit-lions\2018\17\data
Cleaned_Route_Charts\detroit-lions\2018\2\data
Cleaned_Route_Charts\detroit-lions\2018\3\data
Cleaned_Route_Charts\detroit-lions\2018\4\data
Cleaned_Route_Charts\detroit-lions\2018\5\data
Cleaned_Route_Charts\detroit-lions\2018\7\data
Cleaned_Route_Charts\detroit-lions\2018\8\data
Cleaned_Route_Charts\detroit-lions\2018\9\data
Cleaned_Route_Charts\green-bay-packers\2018\1\data
Cleaned_Route_Charts\green-bay-packers\2018\10\data
Cleaned_Route_Charts\green-bay-packers\2018\11\data
Cleaned_Route_Charts\green-bay-packers\2018\12\data
Cleaned_Route_Charts\green-bay-packers\2018\13\data
Cleaned_Route_Charts\green-bay-packers\2018\14\data
Cleaned_Route_Charts\green-bay-packers\2018\15\data
Cleaned_Route_Charts\green-bay-packers\2018\16\data
Cleaned_Route_Charts\green-bay-packers\2018\17\data
Cleaned_Route_Charts\green-bay-packers\2018\2\data
Cleaned_R

Cleaned_Route_Charts\minnesota-vikings\2018\7\data
Cleaned_Route_Charts\minnesota-vikings\2018\8\data
Cleaned_Route_Charts\minnesota-vikings\2018\9\data
Cleaned_Route_Charts\new-england-patriots\2018\1\data
Cleaned_Route_Charts\new-england-patriots\2018\10\data
Cleaned_Route_Charts\new-england-patriots\2018\12\data
Cleaned_Route_Charts\new-england-patriots\2018\13\data
Cleaned_Route_Charts\new-england-patriots\2018\14\data
Cleaned_Route_Charts\new-england-patriots\2018\15\data
Cleaned_Route_Charts\new-england-patriots\2018\16\data
Cleaned_Route_Charts\new-england-patriots\2018\17\data
Cleaned_Route_Charts\new-england-patriots\2018\2\data
Cleaned_Route_Charts\new-england-patriots\2018\3\data
Cleaned_Route_Charts\new-england-patriots\2018\4\data
Cleaned_Route_Charts\new-england-patriots\2018\5\data
Cleaned_Route_Charts\new-england-patriots\2018\6\data
Cleaned_Route_Charts\new-england-patriots\2018\7\data
Cleaned_Route_Charts\new-england-patriots\2018\8\data
Cleaned_Route_Charts\new-engla

Cleaned_Route_Charts\tampabay-buccaneers\2018\15\data
Cleaned_Route_Charts\tampabay-buccaneers\2018\16\data
Cleaned_Route_Charts\tampabay-buccaneers\2018\17\data
Cleaned_Route_Charts\tampabay-buccaneers\2018\2\data
Cleaned_Route_Charts\tampabay-buccaneers\2018\3\data
Cleaned_Route_Charts\tampabay-buccaneers\2018\4\data
Cleaned_Route_Charts\tampabay-buccaneers\2018\6\data
Cleaned_Route_Charts\tampabay-buccaneers\2018\7\data
Cleaned_Route_Charts\tampabay-buccaneers\2018\8\data
Cleaned_Route_Charts\tampabay-buccaneers\2018\9\data
Cleaned_Route_Charts\tennessee-titans\2018\1\data
Cleaned_Route_Charts\tennessee-titans\2018\10\data
Cleaned_Route_Charts\tennessee-titans\2018\11\data
Cleaned_Route_Charts\tennessee-titans\2018\12\data
Cleaned_Route_Charts\tennessee-titans\2018\13\data
Cleaned_Route_Charts\tennessee-titans\2018\14\data
Cleaned_Route_Charts\tennessee-titans\2018\15\data
Cleaned_Route_Charts\tennessee-titans\2018\16\data
Cleaned_Route_Charts\tennessee-titans\2018\17\data
Cleaned_R